In [2]:
from __future__ import division, print_function
from keras.models import Sequential
from keras.models import load_model
from keras.layers.core import Activation, Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.optimizers import Adam
from scipy.misc import imresize
import collections
import numpy as np
import os

import wrapped_game as wp

Using TensorFlow backend.


# DQN - Deep Q-Learning

<img src="DQN.png" />

## Pré processamento

A entrada vem em um conjunto de quatro 800 x 800 imagens, então a forma da entrada é (4, 800, 800). No entanto, a rede espera sua entrada como um tensor de forma quadrangular (tamanho do lote, 80, 80, 4). No começo do jogo, não temos quatro quadros, portanto, falsificamos o empilhamento do primeiro quadro quatro vezes. A forma do tensor de saída retornado dessa função é (80, 80, 4).


A única diferença é o tamanho da entrada e da saída. Nossa forma de entrada é (80, 80, 4) enquanto a deles (Deep Mind) era (84, 84, 4) e nossa saída é (3) correspondente às três ações para as quais o valor da função Q precisa ser computado, enquanto que elas foram (18), correspondente às ações possíveis da Atari.

In [ ]:
def preprocess_images(images):
    if images.shape[0] < 4:
        # 1 imagem
        x_t = images[0]
        x_t = imresize(x_t, (80, 80))
        x_t = x_t.astype("float")
        x_t /= 255.0
        s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    else:
        # 4 imagens
        xt_list = []
        for i in range(images.shape[0]):
            x_t = imresize(images[i], (80, 80))
            x_t = x_t.astype("float")
            x_t /= 255.0
            xt_list.append(x_t)
        s_t = np.stack((xt_list[0], xt_list[1], xt_list[2], xt_list[3]), 
                       axis=2)
    s_t = np.expand_dims(s_t, axis=0)
    return s_t

In [ ]:
def get_next_batch(experience, model, num_actions, gamma, batch_size):
    
    #amostra aleatória
    batch_indices = np.random.randint(low=0, high=len(experience), size=batch_size)
        
    #inicializa X e Y e o batch de acordo com os indices
    
    batch = [experience[i] for i in batch_indices]
    
    X = np.zeros((batch_size, 80, 80, 4))
    Y = np.zeros((batch_size, num_actions))
    
    for i in range(len(batch)):
        s_t, a_t, r_t, s_tp1, game_over = batch[i]
        X[i] = s_t
        Y[i] = model.predict(s_t)[0]
        Q_sa = np.max(model.predict(s_tp1)[0])
        if game_over:
            Y[i, a_t] = r_t
        else:
            Y[i, a_t] = r_t + gamma * Q_sa
    return X, Y

### Inicializa os parâmetros

Cada época corresponde a um único jogo ou episódio

In [ ]:

DATA_DIR = "../data"
NUM_ACTIONS = 3 # número de ações de saída que a rede pode enviar para o jogo (esquerda, faz nada, direita) (0, 1 e 2)

GAMMA =  # fator de desconto para recompensas futuras
#referem-se a valores iniciais e finais para o parâmetro em exploração de base
INITIAL_EPSILON = 
FINAL_EPSILON = 
MEMORY_SIZE =  # tamanho da mamória de repetição de experiência
NUM_EPOCHS_OBSERVE =  #refere-se ao número de épocas onde a rede tem permissão para explorar o jogo , 
                            #enviando-lhe ações completamente aleatórias e vendo as recompensas
NUM_EPOCHS_TRAIN =  #refere-se ao número de épocas em que a rede será submetida a treinamento on-line


BATCH_SIZE =  #tamanho do mini-lote que usaremos para treinamento
NUM_EPOCHS = NUM_EPOCHS_OBSERVE + NUM_EPOCHS_TRAIN #número total de jogos jogados para o treinamento

### constrói um modelo

Existem três camadas convolucionais e duas camadas totalmente conectadas (Dense). Todas as camadas, exceto a última, possuem a unidade de ativação ReLU. Como estamos prevendo valores de funções Q, temos uma rede de regressão e a última camada não possui unidade de ativação.

A função de perda é a diferença quadrática entre o valor atual de Q (s, a) e seu valor calculado em termos da soma da recompensa e do valor Q descontado Q (s ', a') um passo no futuro, então a função de perda é a de erro quadrado médio (MSE).

Adam -> um bom otimizador de propósito geral, instanciado com uma baixa taxa de aprendizado.

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=8, strides=4,
                 kernel_initializer="normal",
                 padding="same",
                 input_shape=(80, 80, 4)))
model.add(Activation("relu"))
model.add(Conv2D(64, kernel_size=4, strides=2, 
                 kernel_initializer="normal",
                 padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, kernel_size=3, strides=1,
                 kernel_initializer="normal",
                 padding="same"))
model.add(Activation("relu"))
model.add(Flatten())
model.add(Dense(512, kernel_initializer="normal"))
model.add(Activation("relu"))

model.add(Dense(3, kernel_initializer="normal"))

model.compile(optimizer=Adam(lr=1e-6), loss="mse")

###### game, file para salvar, inicializa epsilon 

In [ ]:
game = wp.MyWrappedGame()
experience = collections.deque(maxlen=MEMORY_SIZE)

fout = open("rl-network-results_2.tsv", "w")
num_games, num_wins = 0, 0
epsilon = INITIAL_EPSILON

## Treino

Um jogo corresponde a um único episódio de uma bola que cai do teto e está sendo pego pela pá ou está sendo perdido. A perda é a diferença quadrática entre o valor Q previsto e real para o jogo.

Se estivermos no modo de observação, vamos apenas gerar um número aleatório correspondente a um de nossas ações, caso contrário, usaremos a exploração para selecionar uma ação aleatória ou usar nossa rede neural (que também estamos treinando) para prever a ação que devemos enviar.

Quando a rede é relativamente não treinada, suas previsões não são muito boas, então faz sentido explorar o espaço de estados mais em um esforço para reduzir as chances de ficar preso em um mínimo local. No entanto, à medida que a rede é cada vez mais treinada, reduzimos o valor de forma gradual para que o modelo consiga prever mais e mais ações que a rede envia para o jogo.



In [ ]:
for e in range(NUM_EPOCHS):
    #restabelecemos o estado do jogo neste momento (cada época é um jogo)
    loss = 0.0
    game.reset()

    # pega o primeiro estado
    a_0 = 1  # (0 = esquerda, 1 = faz nada, 2 = direita)
    x_t, r_0, game_over = game.step(a_0) 
    s_t = preprocess_images(x_t)

    while not game_over:
        s_tm1 = s_t #guarda estado atual
        #  próxima ação
        if e <= NUM_EPOCHS_OBSERVE:
            a_t = np.random.randint(low=0, high=NUM_ACTIONS, size=1)[0]
        else:
            if np.random.rand() <= epsilon:
                a_t = np.random.randint(low=0, high=NUM_ACTIONS, size=1)[0]
            else:
                q = model.predict(s_t)[0]
                a_t = np.argmax(q)

        # faz a ação, pega a recompensa
        x_t, r_t, game_over = game.step(a_t)
        s_t = preprocess_images(x_t)
        # se for recompensado, incrementa numero de vitorias
        if r_t == 1:
            num_wins += 1          
                        
        # guarda a experiência
        experience.append((s_tm1, a_t, r_t, s_t, game_over))  #(estado, ação, recompensa, novo estado, fim) 
        
        ### Se memoria cheia, remove o primeiro elemento
        if len(experience) > MEMORY_SIZE:
                experience = experience[1:]

        if e > NUM_EPOCHS_OBSERVE:
            # acaba abservação, começa o treinamento
            # calcula gradiente descendente
            X, Y = get_next_batch(experience, model, NUM_ACTIONS, 
                                      GAMMA, BATCH_SIZE)
            
            loss += model.train_on_batch(X, Y)

    
    # reduz o epsilon
    if epsilon > FINAL_EPSILON:
        epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / NUM_EPOCHS
        
    
    #Prints e Saves
    print("Epoca {:04d}/{:d} | Perda {:.5f} | Vezes ganhas: {:d} | epsilon {:.5f}"
            .format(e + 1, NUM_EPOCHS, loss, num_wins, epsilon))
    fout.write("{:04d}\t{:.5f}\t{:d}\n"
            .format(e + 1, loss, num_wins))
               
    # só para salvar estado atual do modelo em um arquivo               
    if e % 100 == 0:
        model.save("rl-network_2.h5", overwrite=True)
        
        
fout.close()
#salva modelo final pra nao precisar executar novamente
model.save("rl-network_2.h5", overwrite=True)